In [1]:
import numpy as np
import h5py
import dmrghandler.hdf5_io as hdf5_io
import dmrghandler.data_processing as dp
from pathlib import Path
import openpyxl as px
import openpyxl.chart as px_chart
import pandas as pd

In [2]:
catalyst_meta_data = pd.read_csv('catalysis_instances.csv')
# catalyst_meta_data.drop(columns='Unnamed: 0', inplace=True)

In [3]:
catalyst_meta_data.columns

Index(['instance ID', 'molecule', 'geometry', 'basis', 'charge',
       'multiplicity', 'avas_atomic_orbitals', 'avas_minao', 'ref', 'fcidump',
       'AVAS_no_ne', 'log10_hilbert_space_size',
       'log10 Distributed FCI CPUhours', 'Utility Scale',
       'Abs. or Rel. Energy', 'Energy (Hartree)', 'Success Probability'],
      dtype='object')

In [4]:
orig_data_dict_list = [
    {
        'Attempt Result': "Convergence",
        'Calc UUID': "a6d74014-60d7-489b-90a4-42d488a86836",
        'CC Wall Time': "00:08:25",
        'fcidump': "fcidump.30_4a_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
        'fcidump_test_string': "fcidump.30_4a_",
    },
    {
        'Attempt Result': "Bond Dimension Limit Reached",
        'Calc UUID': "53503986-79fe-4eb4-a52c-8a78d2808746",
        'CC Wall Time': "07:12:14",
        'fcidump': "fcidump.31_4a_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
        'fcidump_test_string': "fcidump.31_4a_",
    },
]

    

In [5]:
data_dict_list = []
for data_dict in orig_data_dict_list:
    new_data_dict = {}
    fcidump_test_string = data_dict['fcidump_test_string']
    #Get metadata row
    data_row = catalyst_meta_data[catalyst_meta_data['fcidump'].str.contains(fcidump_test_string)]
    new_data_dict['instance ID'] = data_row['instance ID'].values[0]
    new_data_dict['molecule'] = data_row['molecule'].values[0]
    new_data_dict['charge'] = data_row['charge'].values[0]
    new_data_dict['multiplicity'] = data_row['multiplicity'].values[0]
    new_data_dict['fcidump'] = data_dict['fcidump']
    new_data_dict['log10_hilbert_space_size'] = data_row['log10_hilbert_space_size'].values[0]
    new_data_dict['Attempt Result'] = data_dict['Attempt Result']
    new_data_dict['Calc UUID'] = data_dict['Calc UUID']
    new_data_dict['CC Wall Time'] = data_dict['CC Wall Time']
    if 'Calc UUID Small BD' in data_dict:
        new_data_dict['Calc UUID Small BD'] = data_dict['Calc UUID Small BD']
        new_data_dict['CC Wall Time Small BD'] = data_dict['CC Wall Time Small BD']
    data_dict_list.append(new_data_dict)

    # data_dict.move_to_end('log10_hilbert_space_size', last=False)
    # data_dict.move_to_end('fcidump', last=False)
    # data_dict.move_to_end('multiplicity', last=False)
    # data_dict.move_to_end('charge', last=False)
    # data_dict.move_to_end('molecule', last=False)
    # data_dict.move_to_end('instance ID', last=False)
    
# {
#         'instance ID': "mo_n2_pincer",
#         'molecule': "rc",
#         'charge': "0",
#         'multiplicity': "1",
#         'fcidump': "fcidump.20_rc_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
#         'log10_hilbert_space_size': 13.05419106,
#         'Attempt Result': "Convergence",
#         'Calc UUID': "1a3f8996-3341-4332-b600-954068e54591",
#         'CC Wall Time': "0:43:12",
#         'Calc UUID Small BD': "b425cf4d-62b8-479c-8135-595f880da7c6",
#         'CC Wall Time Small BD': "00:23:30",
#     },

    

In [6]:
data_dict_list

[{'instance ID': 'mo_n2_pincer',
  'molecule': '4a',
  'charge': 0,
  'multiplicity': 4,
  'fcidump': "fcidump.30_4a_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
  'log10_hilbert_space_size': 8.435242662,
  'Attempt Result': 'Convergence',
  'Calc UUID': 'a6d74014-60d7-489b-90a4-42d488a86836',
  'CC Wall Time': '00:08:25'},
 {'instance ID': 'mo_n2_pincer',
  'molecule': '4a',
  'charge': 0,
  'multiplicity': 4,
  'fcidump': "fcidump.31_4a_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
  'log10_hilbert_space_size': 19.84590423,
  'Attempt Result': 'Bond Dimension Limit Reached',
  'Calc UUID': '53503986-79fe-4eb4-a52c-8a78d2808746',
  'CC Wall Time': '07:12:14'}]

In [7]:
# data_dict_list = [
#     {
#         'instance ID': "mo_n2_pincer",
#         'molecule': "rc",
#         'charge': "0",
#         'multiplicity': "1",
#         'fcidump': "fcidump.20_rc_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
#         'log10_hilbert_space_size': 13.05419106,
#         'Attempt Result': "Convergence",
#         'Calc UUID': "1a3f8996-3341-4332-b600-954068e54591",
#         'CC Wall Time': "0:43:12",
#         'Calc UUID Small BD': "b425cf4d-62b8-479c-8135-595f880da7c6",
#         'CC Wall Time Small BD': "00:23:30",
#     },
#     {
#         'instance ID': "mo_n2_pincer",
#         'molecule': "rc",
#         'charge': "0",
#         'multiplicity': "1",
#         'fcidump': "fcidump.21_rc_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
#         'log10_hilbert_space_size': 24.22299965,
#         'Attempt Result': "Bad alloc in DMRG Driver - MemoryError",
#         'Calc UUID': "5aa0be63-2ac1-4c77-9a1c-865fb33dd575",
#         'CC Wall Time': "6:36:32",
#         'Calc UUID Small BD': "625ea1fa-4aad-4f83-a48f-405677652050",
#         'CC Wall Time Small BD': "02:40:38",
#     },
# ]

    

In [8]:

# data_dict = {
#     'instance ID': "mo_n2_pincer",
#     'molecule': "pc",
#     'charge': "0",
#     'multiplicity': "1",
#     'fcidump': "fcidump.26_pc_{'Mo': 'def2-SVP', 'I': 'def2-SVP', 'Cl': 'def2-SVP', 'default': '6-311+G(d,p)'}",
#     'log10_hilbert_space_size': 13.05419106,
#     'Attempt Result': "Convergence",
#     'Calc UUID': "de3c9cf7-4396-4e20-988b-c4f5d4e1cd43",
#     'CC Wall Time': "1:13:16",
# }

In [9]:
excel_storage_path = Path("/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/")

data_file_path = Path("/mnt/d/OneDrive - University of Toronto/zapata/homogeneous_catalysts_dmrg_results_private/")
# data_file_path = data_file_path / Path("1a3f8996-3341-4332-b600-954068e54591") # 20
# data_file_path = data_file_path / Path("5aa0be63-2ac1-4c77-9a1c-865fb33dd575") # 21
# data_file_path = data_file_path / Path("6ff1893b-d72e-40a3-942d-0ddb0559657e") # 24
# data_file_path = data_file_path / Path("e6e51673-7231-4d34-9e51-8c4d887dcf89") # 25
# data_file_path = data_file_path / Path("de3c9cf7-4396-4e20-988b-c4f5d4e1cd43") # 26
# data_file_path = data_file_path / Path("c87fa609-2ffd-4389-8758-b13b53238648") # 27
# data_file_path = data_file_path / Path("9b9f1025-786e-42ba-8918-9aa040013b37") # 28
# data_file_path = data_file_path / Path("75c5729f-cca8-49df-b3f6-8bb797d2eb62") # 29


In [10]:
# print(hdf5_io.show_dataset_hierarchy(data_file))

In [11]:
def add_dmrg_data_chart(
    x_vals_ref,
    y_vals_ref,
    coordinates,
    worksheet,
    x_title="none",
    y_title="none",
    series_name=None,
    chart=None,
    x_min=None,
    x_max=None,
    y_min=None,
    y_max=None,
    marker_symbol="circle",
    marker_fill="156082",
    order=0,
):
    new_chart = False
    if chart is None:
        chart = px_chart.ScatterChart()
        new_chart = True
    chart.style = None
    chart.x_axis.title = x_title
    chart.y_axis.title = y_title

    series = px_chart.Series(values=y_vals_ref, xvalues=x_vals_ref, title=series_name,)
    series.marker.symbol = marker_symbol
    series.marker.graphicalProperties.solidFill = marker_fill # Marker filling
    series.marker.graphicalProperties.line.solidFill = marker_fill # Marker outline
    series.marker.graphicalProperties.shadow = "None" # Marker shadow


    series.graphicalProperties.line.noFill = True

    chart.series.append(series)

    chart.x_axis.scaling.min = x_min
    chart.x_axis.scaling.max = x_max

    chart.y_axis.scaling.min = y_min
    chart.y_axis.scaling.max = y_max

    chart.x_axis.crosses = "min"
    chart.y_axis.crosses = "min"

    if series_name is None:
        chart.legend = None

    if new_chart:
        worksheet.add_chart(chart, coordinates)

    return chart

In [12]:
def add_dmrg_processing_basic(
    workbook,
    dmrg_energies,
    bond_dimensions,
    discarded_weights,
    data_dict,
):
    # Get worksheet name from fcidump name
    # take after . and before {
    fcidump_name = data_dict["fcidump"]
    worksheet_name = fcidump_name.split(".")[1].split("{")[0]

    # Create a new worksheet
    ws = workbook.create_sheet(title=worksheet_name)

    # Add header and corresponding data
    ws.append(list(data_dict.keys()))
    ws.append(list(data_dict.values()))

    ws["J3"] = "Estimated Final Energy"
    ws["M3"] = "DON'T USE THIS FOR COMPARISONS, USE E_DMRG INSTEAD (CELL S3)"
    ws["J4"] = "Finish Reason:"
    ws["J5"] = "Loop Results"

    ws["S3"] = "E_DMRG"
    ws["U3"] = "USE THIS E_DMRG FOR COMPARISONS, Estimated Final Energy IS NOT YET RELIABLE"
    ws["S4"] = "DW Linear Slope"
    ws["S5"] = "DW Linear Intercept"

    data_header_dict = {
        "K": "Loop",
        "L": "DMRG Energy",
        "M": "Bond Dimension",
        "N": "Discarded Weights",
        "O": "1/BD",
        "P": "log10 1/BD",
        "Q": "ln DW",
        "R": "Abs Rel Energy",
        "S": "ln AR Energy",
        "T": "Pred E_DMRG",
        "U": "Pred ln AR Energy",
    }
    ws.append(data_header_dict)

    # Fill loop column
    ws["K7"] = -1
    ws["K8"] = 0
    for i in range(len(dmrg_energies) - 2):
        ws[f"K{7+2+i}"] = i + 1

    # Fill DMRG energy column
    for i, energy in enumerate(dmrg_energies):
        ws[f"L{7+i}"] = energy
        ws[f"L{7+i}"].number_format = "0.0000"

    # Fill bond dimension column
    for i, bond_dim in enumerate(bond_dimensions):
        ws[f"M{7+i}"] = bond_dim

    # Fill discarded weights column
    for i, discarded_weight in enumerate(discarded_weights):
        ws[f"N{7+i}"] = discarded_weight
        ws[f"N{7+i}"].number_format = "0.00E+00"

    # Fill 1/BD column with formula
    for i in range(len(bond_dimensions)):
        ws[f"O{7+i}"] = f"=1/M{7+i}"
        ws[f"O{7+i}"].number_format = "0.00E+00"

    # Fill log10 1/BD column with formula
    for i in range(len(bond_dimensions)):
        ws[f"P{7+i}"] = f"=LOG10(O{7+i})"
        ws[f"P{7+i}"].number_format = "0.0000"

    # Fill ln DW column with formula
    for i in range(len(discarded_weights)):
        ws[f"Q{7+i}"] = f"=LN(N{7+i})"
        ws[f"Q{7+i}"].number_format = "0.0000"

    # Fill Abs Rel Energy column with formula
    for i in range(len(dmrg_energies)):
        ws[f"R{7+i}"] = f"=ABS((L{7+i}-$L$3)/$L$3)"
        ws[f"R{7+i}"].number_format = "0.00E+00"

    # Fill ln AR Energy column with formula
    for i in range(len(dmrg_energies)):
        ws[f"S{7+i}"] = f"=LN(R{7+i})"
        ws[f"S{7+i}"].number_format = "0.0000"

    last_data_row = 7 + len(dmrg_energies) - 1
    # E_DMRG formula
    ws["T3"] = f"=MIN(L7:L{last_data_row})"
    ws["T3"].number_format = "0.0000"

    # linear slope and intercept
    ws["T4"] = f"=SLOPE(S7:S{last_data_row},Q7:Q{last_data_row})"
    ws["T4"].number_format = "0.0000"

    ws["T5"] = f"=INTERCEPT(S7:S{last_data_row},Q7:Q{last_data_row})"
    ws["T5"].number_format = "0.0000"

    # Fill Pred E_DMRG column with formula
    for i in range(len(dmrg_energies)):
        ws[f"T{7+i}"] = f"=ABS($L$3)*EXP($T$5)*N{7+i}^$T$4+($L$3)"
        ws[f"T{7+i}"].number_format = "0.0000"

    # Fill Pred ln AR Energy column with formula
    for i in range(len(dmrg_energies)):
        ws[f"U{7+i}"] = f"=$T$4*Q{7+i}+$T$5"
        ws[f"U{7+i}"].number_format = "0.0000"

    # Put in the estimated final energy
    ws["L3"] = dmrg_energies[-1] - 1e-3
    ws["L3"].number_format = "0.000000"

    E_DMRG_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!L7:L{last_data_row}",
    )

    bond_dim_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!M7:M{last_data_row}",
    )

    chart_row_start = 5

    add_dmrg_data_chart(
        x_vals_ref=bond_dim_ref,
        y_vals_ref=E_DMRG_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="Bond Dimension",
        y_title="DMRG Energy (Hartree)",
        series_name=None,
        chart=None,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
    )

    inv_bond_dim_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!O7:O{last_data_row}",
    )
    chart_row_start += 15
    add_dmrg_data_chart(
        x_vals_ref=inv_bond_dim_ref,
        y_vals_ref=E_DMRG_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="1/(Bond Dimension)",
        y_title="DMRG Energy (Hartree)",
        series_name=None,
        chart=None,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
    )

    log_inv_bond_dim_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!P7:P{last_data_row}",
    )
    chart_row_start += 15
    add_dmrg_data_chart(
        x_vals_ref=log_inv_bond_dim_ref,
        y_vals_ref=E_DMRG_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="log10(1/(Bond Dimension))",
        y_title="DMRG Energy (Hartree)",
        series_name=None,
        chart=None,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
    )

    discarded_weight_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!N7:N{last_data_row}",
    )
    E_DMRG_pred_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!T7:T{last_data_row}",
    )
    chart_row_start += 15
    dw_e_dmrg_chart = add_dmrg_data_chart(
        x_vals_ref=discarded_weight_ref,
        y_vals_ref=E_DMRG_pred_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="Discarded Weight",
        y_title="DMRG Energy (Hartree)",
        series_name="Prediction",
        chart=None,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
        marker_symbol="square",
        marker_fill="00B050",
    )

    add_dmrg_data_chart(
        x_vals_ref=discarded_weight_ref,
        y_vals_ref=E_DMRG_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="Discarded Weight",
        y_title="DMRG Energy (Hartree)",
        series_name="Calculation",
        chart=dw_e_dmrg_chart,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
    )

    ln_discarded_weight_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!Q7:Q{last_data_row}",
    )
    ln_AR_Energy_pred_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!U7:U{last_data_row}",
    )
    ln_AR_Energy_ref = px.chart.Reference(
        worksheet=ws,
        range_string=f"{ws.title}!S7:S{last_data_row}",
    )
    chart_row_start += 15
    dw_e_dmrg_chart = add_dmrg_data_chart(
        x_vals_ref=ln_discarded_weight_ref,
        y_vals_ref=ln_AR_Energy_pred_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="ln(Discarded Weight)",
        y_title="ln(Abs Relative Energy)",
        series_name="Prediction",
        chart=None,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
        marker_symbol="square",
        marker_fill="00B050",
    )

    add_dmrg_data_chart(
        x_vals_ref=ln_discarded_weight_ref,
        y_vals_ref=ln_AR_Energy_ref,
        coordinates=f"A{chart_row_start}",
        worksheet=ws,
        x_title="ln(Discarded Weight)",
        y_title="ln(Abs Relative Energy)",
        series_name="Calculation",
        chart=dw_e_dmrg_chart,
        x_min=None,
        x_max=None,
        y_min=None,
        y_max=None,
    )

In [13]:
wb = px.Workbook()

for data_dict in data_dict_list:
    data_file = data_file_path / Path(data_dict['Calc UUID']) / Path("dmrg_results.hdf5")
    # Get DMRG energy, bond dimensions, and truncation error for each loop
    (
        dmrg_energies,
        bond_dimensions,
        discarded_weights,
        num_loops,
        num_dmrg_calculations,
    ) = dp.get_data_from_incomplete_processing(data_file)

    if 'Calc UUID Small BD' in data_dict:
        data_file_small_bd = data_file_path / Path(data_dict['Calc UUID Small BD']) / Path("dmrg_results.hdf5")
        (
        dmrg_energies_small_bd,
        bond_dimensions_small_bd,
        discarded_weights_small_bd,
        num_loops_small_bd,
        num_dmrg_calculations_small_bd,
        ) = dp.get_data_from_incomplete_processing(data_file_small_bd)
        dmrg_energies = np.hstack((dmrg_energies_small_bd, dmrg_energies))
        bond_dimensions = np.hstack((bond_dimensions_small_bd, bond_dimensions))
        discarded_weights = np.hstack((discarded_weights_small_bd, discarded_weights))
        num_loops = num_loops_small_bd + num_loops
        num_dmrg_calculations = num_dmrg_calculations_small_bd + num_dmrg_calculations


    add_dmrg_processing_basic(
        workbook=wb,
        dmrg_energies=dmrg_energies,
        bond_dimensions=bond_dimensions,
        discarded_weights=discarded_weights,
        data_dict=data_dict,
    )
wb.save(excel_storage_path/'sample.xlsx')

Last loop included = 24
Processed results available
Checking that processed results match raw results.
Last loop included = 44
Processed results available
Checking that processed results match raw results.


In [14]:
print(f"Calculation: {data_file.parent.name}")
print("There are a total of", len(dmrg_energies), "DMRG calculations.")
print("Last loop included =", num_loops)
assert len(dmrg_energies) == num_loops + 2
print("DMRG energies:")
for value in dmrg_energies:
    print(value)
print("Bond dimensions:")
for value in bond_dimensions:
    print(value)
print("Discarded weights:")
for value in discarded_weights:
    print(value)

Calculation: 53503986-79fe-4eb4-a52c-8a78d2808746
There are a total of 46 DMRG calculations.
Last loop included = 44
DMRG energies:
-5103.750551172535
-5103.749648114455
-5103.770524909578
-5103.78259310729
-5103.7914003493315
-5103.789650404845
-5103.800399801571
-5103.811057747357
-5103.814146899301
-5103.817815455213
-5103.824708128856
-5103.828055005798
-5103.833160287367
-5103.836630063503
-5103.841563898539
-5103.8477048375635
-5103.850125437115
-5103.8548824534155
-5103.85994089363
-5103.861601382936
-5103.862868835591
-5103.866009390604
-5103.867322944848
-5103.870011273531
-5103.872653065163
-5103.874469206422
-5103.875178494839
-5103.8771767940925
-5103.877848159044
-5103.878621958424
-5103.879352144496
-5103.880205573198
-5103.881054034138
-5103.88165339447
-5103.882192921025
-5103.8827154354685
-5103.883180380613
-5103.8836639740675
-5103.884007361787
-5103.884460377254
-5103.884774739019
-5103.885061821744
-5103.885372741653
-5103.885651174251
-5103.885882701058
-5103.8860